In [8]:
df = read.csv("../Data/newSeoul2019.csv")
head(df)

년도,월,자치구명,발생건수,사망자수,부상자수
2011,1,종로구,115,2,165
2011,2,종로구,99,0,134
2011,3,종로구,116,0,162
2011,4,종로구,110,1,153
2011,5,종로구,122,0,157
2011,6,종로구,116,3,163


In [16]:
library(caret) #sample
library(nnet) #인공신경망 , 다항 로지스틱 회귀분석
library(e1071) # svm
library(randomForest) # randomForest
library(party) # 의사결정나무 ctree

Loading required package: lattice
Loading required package: ggplot2
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:ggplot2’:

    margin

Loading required package: grid
Loading required package: mvtnorm
Loading required package: modeltools
Loading required package: stats4
Loading required package: strucchange
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Loading required package: sandwich


In [11]:

df = df[df$년도 == 2019,]

In [13]:
head(df)

,년도,월,자치구명,발생건수,사망자수,부상자수
2401,2019,1,종로구,87,1,125
2402,2019,2,종로구,66,1,84
2403,2019,3,종로구,87,2,122
2404,2019,4,종로구,85,0,131
2405,2019,5,종로구,112,1,158
2406,2019,6,종로구,95,1,116


In [15]:
rownames(df) = c()
head(df)

년도,월,자치구명,발생건수,사망자수,부상자수
2019,1,종로구,87,1,125
2019,2,종로구,66,1,84
2019,3,종로구,87,2,122
2019,4,종로구,85,0,131
2019,5,종로구,112,1,158
2019,6,종로구,95,1,116


In [44]:
df = subset(df, select = c('발생건수','부상자수'))
head(df)

발생건수,부상자수
87,125
66,84
87,122
85,131
112,158
95,116


In [23]:
?predict

In [31]:
round(1.5)

[1] 2

In [58]:


# 표준화
#iris2 <- iris
#iris2[,1:4] <- as.data.frame(scale(iris[,1:4]))

# 샘플나누기

sampling <- function(){
samp <<- createDataPartition(df$부상자수 , p = 0.7,list = F)
data.tr <<- df[samp,]
data.test <<- df[-samp,]

x <<- subset(data.test, select = -부상자수)
y <<- data.test$부상자수
    
}


sampling() 
head(samp)

#인공신경망 테스트
ANN <- function(data.tr , data.test , x , y ){
    model.nnet <- nnet(부상자수 ~. , data = data.tr , size = 100 , linout=T)
    pred <- predict(model.nnet , x , type = "raw")
#     cat("예측력" ,mean(pred == y), "입니다.")
#     table(pred,y)
#     return(mean(pred == y))
    return(1 - (sum((y - pred)^2) / sum((y - mean(y))^2)))
}


ANN(data.tr , data.test , x , y )

#의사결정 나무 
CTREE <-function(data.tr , data.test , x , y ){
    model.ctree <- ctree(부상자수 ~ . , data=data.tr)
    pred <- predict(model.ctree , x )
#     cat("예측력" ,mean(pred == y), "입니다.")
#     table(pred,y)
#     return(mean(round(pred) == y))
    return(1 - (sum((y - pred)^2) / sum((y - mean(y))^2)))
}

 CTREE(data.tr , data.test , x , y )

#RandomForesst
RandomForest<-function(data.tr , data.test , x , y ){
    model.rf <- randomForest(부상자수 ~ . , data=data.tr, ntree = 100)
    pred <- predict(model.rf , x )
#     cat("예측력" ,mean(pred == y), "입니다.")
#     table(pred,y)
#     return(mean(round(pred) == y))
    
    return(1 - (sum((y - pred)^2) / sum((y - mean(y))^2)))
} 




#SVM
SVM <-function(data.tr , data.test , x , y ){
    model.svm <- svm(부상자수 ~ . , data=data.tr)
    pred <- predict(model.svm , x )
#     cat(" SVM 예측력" ,mean(round(pred) == y), "입니다.")
#     cat(round(pred) , y)
#     table(pred,y)
#     return(mean(round(pred) == y))
      return(1 - (sum((y - pred)^2) / sum((y - mean(y))^2)))
} 

#Logistic regression
Logisticreg <-function(data.tr , data.test , x , y ){
    model.log <- multinom(부상자수 ~ . , data=data.tr)
    pred <- predict(model.log , x )
#     cat("예측력" ,mean(pred == y), "입니다.")
    table(pred,y)
    return(mean(pred == y))
#     return(1 - (sum((y - pred)^2) / sum((y - mean(y))^2)))
} 

a <- rep(0,4)
iteration <- 5
for(i in 1:iteration){
    
    a = a + c(
            ANN(data.tr , data.test , x , y ),
            CTREE(data.tr , data.test , x , y ),
            RandomForest(data.tr , data.test , x , y ),
            SVM(data.tr , data.test , x , y )
    )
    
}
a<- a/iteration

result<-data.frame(예측력 = a)
rownames(result) = c("인공신경망","의사결정나무","RandomForest","SVM")
result



Resample1
3
4
5
6
7
8


# weights:  301
initial  value 7686471.985477 
iter  10 value 1239142.381076
iter  20 value 412833.791011
iter  30 value 347455.448460
iter  40 value 240846.719196
iter  50 value 90042.628502
iter  60 value 50226.585110
iter  70 value 40196.929854
iter  80 value 39843.179300
iter  90 value 39711.465553
iter 100 value 39680.038300
final  value 39680.038300 
stopped after 100 iterations


[1] 0.9761512

[1] 0.9318831

# weights:  301
initial  value 7496366.960985 
iter  10 value 1231316.494653
iter  20 value 1162883.763019
iter  30 value 415204.021774
iter  40 value 195742.080375
iter  50 value 175345.573745
iter  60 value 166384.968289
iter  70 value 89423.622701
iter  80 value 51472.658001
iter  90 value 46300.778071
iter 100 value 45263.031762
final  value 45263.031762 
stopped after 100 iterations
# weights:  301
initial  value 8117661.456336 
iter  10 value 1237454.037183
iter  20 value 1181607.200741
iter  30 value 373819.936381
iter  40 value 264402.235440
iter  50 value 235270.298093
iter  60 value 153440.311060
iter  70 value 106616.012801
iter  80 value 91282.408599
iter  90 value 68908.951836
iter 100 value 54500.170859
final  value 54500.170859 
stopped after 100 iterations
# weights:  301
initial  value 7903120.042804 
iter  10 value 1224095.352990
iter  20 value 365731.911800
iter  30 value 228468.451834
iter  40 value 102343.745552
iter  50 value 42868.495194
iter  60 value 40840.8913

,예측력
인공신경망,0.9683472
의사결정나무,0.9318831
RandomForest,0.9637853
SVM,0.9259185


In [59]:
    model.nnet <- nnet(부상자수 ~. , data = data.tr , size = 100 , linout=T)
    pred <- predict(model.nnet , x , type = "raw")
#     cat("예측력" ,mean(pred == y), "입니다.")
    table(pred,y)
#     return(mean(pred == y))
    return(1 - (sum((y - pred)^2) / sum((y - mean(y))^2)))

# weights:  301
initial  value 8005902.356042 
iter  10 value 1214981.743931
iter  20 value 1097710.367532
iter  30 value 574245.415819
iter  40 value 374587.691573
iter  50 value 132790.191544
iter  60 value 43843.923762
iter  70 value 39783.848289
iter  80 value 39324.650259
iter  90 value 39288.708436
iter 100 value 39250.531981
final  value 39250.531981 
stopped after 100 iterations


                  y
pred               84 90 91 92 93 94 96 99 100 102 103 110 113 115 118 123 125
  85.7570863492349  0  0  1  0  0  0  0  0   0   0   0   0   0   0   0   0   0
  91.0431013533615  1  1  0  0  0  0  0  0   0   0   0   0   0   0   0   0   0
  96.3901439944536  0  0  0  0  0  0  1  0   0   0   0   0   0   0   0   0   0
  97.7347208570231  0  0  0  0  1  0  0  0   0   0   0   0   0   0   0   0   0
  99.0819639126916  0  0  0  0  0  0  0  0   0   1   0   0   0   0   0   0   0
  101.783530758869  0  0  0  0  0  1  0  0   0   0   0   0   0   0   0   0   0
  103.13739784785   0  0  0  1  0  0  0  0   0   0   1   0   0   0   0   0   0
  107.208622782628  0  0  0  0  0  0  0  0   1   0   0   0   0   0   0   0   0
  111.289635236104  0  0  0  0  0  0  0  0   0   0   0   1   0   1   0   0   0
  112.651140037672  0  0  0  0  0  0  0  1   0   0   0   0   0   0   0   0   0
  116.73625402691   0  0  0  0  0  0  0  0   0   0   0   0   1   0   0   0   0
  119.458186304629  0  0  0  0  

[1] 0.9715864

In [61]:
    model.nnet <- nnet(부상자수 ~. , data = data.tr , size = 100 )
    pred <- predict(model.nnet , x , type = "raw")
#     cat("예측력" ,mean(pred == y), "입니다.")

    table(pred,y)
#     return(mean(pred == y))
    return(1 - (sum((y - pred)^2) / sum((y - mean(y))^2)))

# weights:  301
initial  value 7816636.105551 
final  value 7746229.000000 
converged
    [,1]
1      1
2      1
10     1
15     1
19     1
22     1
24     1
25     1
30     1
31     1
32     1
33     1
35     1
39     1
41     1
44     1
45     1
46     1
47     1
51     1
55     1
57     1
60     1
65     1
70     1
73     1
75     1
77     1
78     1
79     1
81     1
83     1
90     1
91     1
95     1
107    1
117    1
120    1
122    1
123    1
126    1
132    1
134    1
137    1
146    1
150    1
152    1
159    1
160    1
170    1
173    1
176    1
177    1
193    1
197    1
199    1
200    1
201    1
202    1
204    1
207    1
213    1
219    1
221    1
222    1
226    1
229    1
232    1
239    1
246    1
250    1
254    1
257    1
258    1
259    1
263    1
269    1
273    1
275    1
278    1
279    1
281    1
283    1
286    1
287    1
288    1
291    1
293    1
295    1


    y
pred 84 90 91 92 93 94 96 99 100 102 103 110 113 115 118 123 125 126 127 131
   1  1  1  1  1  1  1  1  1   1   1   1   1   1   1   2   2   3   1   1   1
    y
pred 133 134 135 136 139 140 141 142 144 145 148 149 150 151 153 154 163 164
   1   1   2   2   2   1   1   1   2   1   1   1   1   1   2   1   1   1   1
    y
pred 169 172 173 175 177 178 179 180 182 187 189 192 193 194 217 219 237 252
   1   1   1   3   1   2   2   2   2   1   1   2   1   1   1   1   1   1   1
    y
pred 253 277 288 293 296 299 312 325 330 336 344 371 389 398 447 484 542
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1

[1] -4.020362